In [1]:
import torch

In [2]:
img = torch.rand(1 , 3 , 256 ,256)

In [4]:
from Vit import Patch_Embed

Patching = Patch_Embed(p = 8 , proj_dim= 1024 , max_patches=100000)

patches = Patching(img)

In [5]:
patches.shape

torch.Size([1, 1024, 1024])

In [7]:
import torch.nn as nn
from torch.nn.init import xavier_uniform_

input_dim = patches.shape[-1]
out_dim = patches.shape[-1]
w_q = nn.Parameter(torch.empty(input_dim , out_dim))
w_k = nn.Parameter(torch.empty(input_dim , out_dim))
w_v = nn.Parameter(torch.empty(input_dim , out_dim))

xavier_uniform_(w_q)
xavier_uniform_(w_k)
xavier_uniform_(w_v)

Parameter containing:
tensor([[-0.0399, -0.0415,  0.0390,  ...,  0.0262,  0.0222, -0.0488],
        [ 0.0466, -0.0438, -0.0329,  ...,  0.0476,  0.0432,  0.0539],
        [ 0.0541,  0.0161,  0.0070,  ...,  0.0276,  0.0327, -0.0500],
        ...,
        [-0.0189, -0.0368, -0.0013,  ..., -0.0228, -0.0374, -0.0113],
        [-0.0503,  0.0048, -0.0493,  ..., -0.0137,  0.0532, -0.0158],
        [ 0.0187,  0.0066, -0.0149,  ...,  0.0124,  0.0443, -0.0094]],
       requires_grad=True)

In [22]:
V = patches @ w_v.to('cuda')
K = patches @ w_k.to('cuda')
Q = patches @ w_q.to('cuda')
dk = torch.tensor(K.shape[-1], dtype=torch.float32)
V.shape , K.shape , Q.shape

(torch.Size([1, 1024, 1024]),
 torch.Size([1, 1024, 1024]),
 torch.Size([1, 1024, 1024]))

In [25]:
scores  = Q @ K.transpose(-2 , -1)
scores = scores / torch.sqrt(dk)
attention_scores = torch.softmax(scores , dim = -1)
print(attention_scores.shape)
out = attention_scores @ V

torch.Size([1, 1024, 1024])


In [26]:
out.shape

torch.Size([1, 1024, 1024])